In [1]:
!pip install --quiet cirq

     |████████████████████████████████| 1.8MB 6.0MB/s 
     |████████████████████████████████| 1.3MB 37.9MB/s 


In [6]:
# Deutsch algorithm
import cirq
import time
import numpy as np

function=0
time_result = []
for n in range(3,28):
  q = [cirq.NamedQubit("a"+str(i)) for i in range(n)]
  circuit = cirq.Circuit()
  moments = [cirq.H(q[i]) for i in range(n)]
  if function!=None:
      for i in range(n-1):
        moments += [cirq.CNOT(q[i], q[-1])]

  moments += [cirq.H(q[i]) for i in range(n-1)]
  circuit.append(moments)

  simulator = cirq.Simulator()
  initial_state = np.array([0]*(2**n))
  initial_state[1] = 1
  start_time = time.time()
  results = simulator.simulate(circuit, initial_state=initial_state, qubit_order=q)
  time_result.append((time.time() - start_time))

In [7]:
time_result

[0.002010822296142578,
 0.006912946701049805,
 0.007322072982788086,
 0.005489826202392578,
 0.004613637924194336,
 0.0051419734954833984,
 0.006871938705444336,
 0.006784915924072266,
 0.0053822994232177734,
 0.006264925003051758,
 0.007646083831787109,
 0.010567426681518555,
 0.015895366668701172,
 0.026006460189819336,
 0.04778003692626953,
 0.08736705780029297,
 0.19181537628173828,
 0.40337467193603516,
 0.8255362510681152,
 1.6467773914337158,
 3.2954633235931396,
 6.86741828918457,
 14.152146100997925,
 29.098552465438843,
 60.068238258361816]

#Supremacy circuits simulation

In [8]:
import numpy as np
import cirq
import time
from cirq.experiments.google_v2_supremacy_circuit import *
def experiment1(n, m, cz_depth, seed, qubits_to_be_ignored=None):
  # qubits = [cirq.GridQubit(i,j) for i in range(n) for j in range(n)]
  circuit = generate_boixo_2018_supremacy_circuits_v2_grid(n_rows=n, n_cols=m, cz_depth=cz_depth, seed=seed)
  supremacy_circuit = cirq.Circuit()
  for moment in circuit:
    for operation in moment:
      if not set(qubits_to_be_ignored) & set(operation.qubits):
        supremacy_circuit.append(operation)
  s = n*m-len(qubits_to_be_ignored)
  initial_state_vector = np.array([0]*2**s)
  initial_state_vector[0] = 1
  time_result = []
  qubits = list(supremacy_circuit.all_qubits())
  for p in range(10):
    start_time = time.time()
    simulator = cirq.Simulator()
    results = simulator.simulate(supremacy_circuit, initial_state = initial_state_vector, qubit_order = qubits)
    time_result.append(time.time() - start_time)
  time_result = np.array(time_result)
  return supremacy_circuit, results, np.mean(time_result)


In [9]:
import numpy as np
from itertools import product
from random import sample
time_results = []
c, r, mt = experiment1(n=2, m=2, cz_depth=4, seed=2, qubits_to_be_ignored=[])

time_results.append(mt)

for n in range(3,6):
  list_of_indices = list(product(range(n),repeat=2))
  for i in range(2*n-2, -1,-1):
    qubits_to_be_ignored = [cirq.GridQubit(x[0], x[1]) for x in sample(list_of_indices, i)]
    circuit, result, x = experiment1(n=n, m=n, cz_depth=2*n,seed=2, qubits_to_be_ignored=qubits_to_be_ignored)
  # circuit, result, x = experiment1(n=n, m=n, cz_depth=2*n,seed=2)
    time_results.append(x)

c, r, mt = experiment1(n=7, m=4, cz_depth=14, seed=1, qubits_to_be_ignored=[cirq.GridQubit(0,0), cirq.GridQubit(0,1)])

time_results.append(mt)

c, r, mt = experiment1(n=7, m=4, cz_depth=14, seed=1, qubits_to_be_ignored=[cirq.GridQubit(0,0)])

time_results.append(mt)
time_results

4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


[0.004195451736450195,
 0.0038480520248413085,
 0.005486869812011718,
 0.004797840118408203,
 0.006438493728637695,
 0.008353686332702637,
 0.008139204978942872,
 0.010401463508605957,
 0.011721992492675781,
 0.01622469425201416,
 0.021231603622436524,
 0.030955195426940918,
 0.05592355728149414,
 0.20605340003967285,
 0.34598114490509035,
 0.5079902172088623,
 1.0100622415542602,
 2.0481415510177614,
 4.197118926048279,
 8.385995697975158,
 16.85278730392456,
 36.125806427001955,
 94.51600887775422,
 198.4059149980545]

# Multiplier simulation

In [ ]:
# from google.colab import  drive
# drive.mount('/content/drive')

In [ ]:
# %cd drive/My\ Drive/

In [ ]:
# %cd quantify

In [ ]:
# !cp -avr mathematics/ /usr/local/lib/python3.6/dist-packages

In [22]:
# make sure to import the mathematics package from quantify
import cirq
import mathematics.multiplier as mt
import numpy as np
import time

def multiplier_experiment(n, qubits_to_be_excluded=[]):

    # A and B are the numbers to be multiplied
    A = [cirq.NamedQubit('a' + str(i)) for i in range(n)]
    B = [cirq.NamedQubit('b' + str(i)) for i in range(n)]

    # instanciate the multiplier class and invoke the multiply method
    # the result is the corresponding circuit
    circuit = mt.multiplier(A, B).multiply()
    final_circuit = cirq.Circuit()
    for moment in circuit:
      for operation in moment:
        if not set(qubits_to_be_ignored) & set(operation.qubits):
          final_circuit.append(operation)
    
    
    simulator = cirq.Simulator()

    # sort the qubits in a way such that B has the most significant bits then A then P
    
    qubits = sorted(list(final_circuit.all_qubits()))[::-1]

    s = len(qubits)
    print(s)
    intial_state = [0] * 2 ** (len(qubits))

    intial_state[0] = 1
    intial_state = np.array(intial_state, dtype=np.complex64)
    start_time = time.time()
    result = simulator.simulate(final_circuit, qubit_order=qubits, initial_state=intial_state)
    time_result = time.time()-start_time

    # """Now consider B and A as a concatinated bitstring with B having the most significant bits
    #     Afterwards add 2n+1 bits of 0 which represents P0, P1...P2n
    # """

    # """the list of all possible integers is exactly the range(2^(2n)) but we need to shift it 
    #     in order to initialize the qubits of P to 0
    # """
    # shift = 2 ** n - 1
    # # list_of_possible_integers = [i << shift for i in range(2 ** (2 * n))]
    # list_of_possible_integers = [2 ** (2 * n)-2]
    # # print(list_of_possible_integers)
    # for i in list_of_possible_integers:
    #     intial_state[i] = 1
    #     # state[1920]=1
    #     intial_state = np.array(intial_state, dtype=np.complex64)
    #     start_time = time.time()
    #     result = simulator.simulate(circuit, qubit_order=qubits, initial_state=intial_state)
    #     time_result = time.time()-start_time
    #     BA = i >> shift
    #     B_times_A = BA % (2 ** 3) * (BA >> n)
    #     # assert (np.where(result.final_state == 1)[0][0] == i + B_times_A)
    #     intial_state[i] = 0
    return time_result

In [23]:
import numpy as np
from itertools import product
from random import sample
time_results = []
time_results.append(multiplier_experiment(3, qubits_to_be_excluded=[]))
for n in range(4,8):
  # list_of_indices = list(product(range(n),repeat=2))
  if n == 7:
    for i in range(3,1,-1):
      qubits_to_be_ignored = [cirq.NamedQubit("P"+str(j)) for j in range(i)]
      time_results.append(multiplier_experiment(n, qubits_to_be_excluded=qubits_to_be_ignored))
  else:
    for i in range(3,-1,-1):
      qubits_to_be_ignored = [cirq.NamedQubit("P"+str(j)) for j in range(i)]
      time_results.append(multiplier_experiment(n, qubits_to_be_excluded=qubits_to_be_ignored))


12
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [24]:
time_results

[0.011724710464477539,
 0.021884679794311523,
 0.028303146362304688,
 0.030359268188476562,
 0.03738212585449219,
 0.1295318603515625,
 0.25028014183044434,
 0.40126848220825195,
 0.7607660293579102,
 2.0643651485443115,
 4.024325132369995,
 7.521160364151001,
 15.930748462677002,
 39.37988615036011,
 75.10322833061218]

#simulation with qsim

In [15]:
!pip install --quiet qsimcirq

In [16]:
import cirq
import qsimcirq
import time
from cirq.experiments.google_v2_supremacy_circuit import *

def experiment2(n, m, cz_depth, seed, qubits_to_be_ignored=None):
  # qubits = [cirq.GridQubit(i,j) for i in range(n) for j in range(n)]
  circuit = generate_boixo_2018_supremacy_circuits_v2_grid(n_rows=n, n_cols=m, cz_depth=cz_depth, seed=seed)
  supremacy_circuit = cirq.Circuit()
  qubit_to_be_ignored = cirq.GridQubit(0,0)
  supremacy_circuit = cirq.Circuit()
  for moment in circuit:
    for operation in moment:
      if not set(qubits_to_be_ignored) & set(operation.qubits):
        supremacy_circuit.append(operation)
  s = n*m-len(qubits_to_be_ignored)
  qubits = list(supremacy_circuit.all_qubits())
  initial_state_vector = np.array([0]*2**s, dtype=np.complex64)
  initial_state_vector[0] = 1
  time_result = []
  for p in range(10):
    start_time = time.time()
    simulator = qsimcirq.QSimSimulator()
    results = simulator.simulate(supremacy_circuit, initial_state = initial_state_vector, qubit_order = qubits)
    time_result.append(time.time() - start_time)
  time_result = np.array(time_result)
  return circuit, results, np.mean(time_result)

In [ ]:
time_results_qsim = []
for n in range(2,6):
  circuit, result, x = experiment2(n=n, m=n, cz_depth=2*n, seed=2)
  time_results_qsim.append(x)

c, r, mt = experiment2(n=4, m=7, cz_depth=2*n, seed=2)
time_results_qsim.append(mt)
time_results_qsim

[0.0031584739685058595,
 0.0022501230239868166,
 0.005138659477233886,
 0.9665431261062623,
 8.048966884613037]

In [12]:
import numpy as np
from itertools import product
from random import sample
time_results = []
c, r, mt = experiment2(n=2, m=2, cz_depth=4, seed=2, qubits_to_be_ignored=[])

time_results.append(mt)

for n in range(3,6):
  list_of_indices = list(product(range(n),repeat=2))
  for i in range(2*n-2, -1,-1):
    qubits_to_be_ignored = [cirq.GridQubit(x[0], x[1]) for x in sample(list_of_indices, i)]
    circuit, result, x = experiment2(n=n, m=n, cz_depth=2*n,seed=2, qubits_to_be_ignored=qubits_to_be_ignored)
  # circuit, result, x = experiment1(n=n, m=n, cz_depth=2*n,seed=2)
    time_results.append(x)

c, r, mt = experiment2(n=7, m=4, cz_depth=14, seed=1, qubits_to_be_ignored=[cirq.GridQubit(0,0), cirq.GridQubit(0,1)])

time_results.append(mt)

c, r, mt = experiment2(n=7, m=4, cz_depth=14, seed=1, qubits_to_be_ignored=[cirq.GridQubit(0,0)])

time_results.append(mt)
time_results

[0.0036581993103027345,
 0.001595783233642578,
 0.0019182443618774414,
 0.00233311653137207,
 0.0025788545608520508,
 0.0027222633361816406,
 0.002931499481201172,
 0.0032016992568969726,
 0.00416877269744873,
 0.003957128524780274,
 0.004674053192138672,
 0.005312824249267578,
 0.007239437103271485,
 0.009012174606323243,
 0.012385129928588867,
 0.02064509391784668,
 0.04414701461791992,
 0.10217959880828857,
 0.2147991418838501,
 0.46319141387939455,
 1.031615924835205,
 2.1322256326675415,
 5.9295084238052365,
 12.252466011047364]

In [19]:
import cirq
import mathematics.multiplier as mt
import numpy as np
import time

def multiplier_experiment2(n, qubits_to_be_ignored=[]):

    # A and B are the numbers to be multiplied
    A = [cirq.NamedQubit('a' + str(i)) for i in range(n)]
    B = [cirq.NamedQubit('b' + str(i)) for i in range(n)]

    # instanciate the multiplier class and invoke the multiply method
    # the result is the corresponding circuit
    circuit = mt.multiplier(A, B).multiply()
    final_circuit = cirq.Circuit()
    for moment in circuit:
      for operation in moment:
        if not set(qubits_to_be_ignored) & set(operation.qubits):
          final_circuit.append(operation)
    
    
    simulator = qsimcirq.QSimSimulator()

    # sort the qubits in a way such that B has the most significant bits then A then P
    
    qubits = sorted(list(final_circuit.all_qubits()))[::-1]

    s = len(qubits)
    print(s)
    intial_state = [0] * 2 ** (len(qubits))

    intial_state[0] = 1
    intial_state = np.array(intial_state, dtype=np.complex64)
    start_time = time.time()
    result = simulator.simulate(final_circuit, qubit_order=qubits, initial_state=intial_state)
    time_result = time.time()-start_time

    # """Now consider B and A as a concatinated bitstring with B having the most significant bits
    #     Afterwards add 2n+1 bits of 0 which represents P0, P1...P2n
    # """

    # """the list of all possible integers is exactly the range(2^(2n)) but we need to shift it 
    #     in order to initialize the qubits of P to 0
    # """
    # shift = 2 ** n - 1
    # # list_of_possible_integers = [i << shift for i in range(2 ** (2 * n))]
    # list_of_possible_integers = [2 ** (2 * n)-2]
    # # print(list_of_possible_integers)
    # for i in list_of_possible_integers:
    #     intial_state[i] = 1
    #     # state[1920]=1
    #     intial_state = np.array(intial_state, dtype=np.complex64)
    #     start_time = time.time()
    #     result = simulator.simulate(circuit, qubit_order=qubits, initial_state=intial_state)
    #     time_result = time.time()-start_time
    #     BA = i >> shift
    #     B_times_A = BA % (2 ** 3) * (BA >> n)
    #     # assert (np.where(result.final_state == 1)[0][0] == i + B_times_A)
    #     intial_state[i] = 0
    return time_result

In [20]:
import numpy as np

time_results = []
time_results.append(multiplier_experiment2(3, qubits_to_be_ignored=[]))
for n in range(4,8):
  # list_of_indices = list(product(range(n),repeat=2))
  if n == 7:
    for i in range(3,1,-1):
      qubits_to_be_ignored = [cirq.NamedQubit("P"+str(j)) for j in range(i)]
      time_results.append(multiplier_experiment2(n, qubits_to_be_ignored=qubits_to_be_ignored))
  else:
    for i in range(3,-1,-1):
      qubits_to_be_ignored = [cirq.NamedQubit("P"+str(j)) for j in range(i)]
      time_results.append(multiplier_experiment2(n, qubits_to_be_ignored=qubits_to_be_ignored))
time_results

13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


[0.0043315887451171875,
 0.008138418197631836,
 0.00918126106262207,
 0.022190570831298828,
 0.020337343215942383,
 0.0599520206451416,
 0.0978083610534668,
 0.2455437183380127,
 0.5239322185516357,
 1.3616175651550293,
 2.809159994125366,
 5.8240745067596436,
 11.742754459381104,
 28.574852466583252,
 58.392494916915894]